This notebook shows how to pre-process and invert ICESat-2 ATL15 data over subglacial lakes.

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '../source')
sys.path.insert(0, '../scripts')

First step: Choose a lake name from the Siegfried & Fricker (2018) inventory 

In [ ]:
from load_lakes import gdf
print(gdf['name'].to_list())

In [ ]:
import metavars
metavars.lake_name = 'synth'
lake_name = metavars.lake_name

You can also set lake_name = 'synth' for the synthetic example

Next, define paths to data directories (modify as necessary!)

See README for links to download the data

In [ ]:
paths = {}
paths['icesat'] = '../../../ICESat-2/ATL15/ATL15_AA_0314_01km_002_02.nc'
paths['wavi'] = '../../../WAVI/WAVI_5km_Initial.nc'
paths['measures'] = '../../../measures/antarctic_ice_vel_phase_map_v01.nc'
paths['bedmachine'] = '../../../bedmachine/BedMachineAntarctica-v3.nc'

Next we pre-process the ICESat-2 data for use in the inversion... this can take several (15+) minutes!

This step is skipped if the data already exists.

You can edit certain aspects of the pre-processing, like the size of the bounding box around the lake, by modifying proc_icesat_data.py

In [ ]:
from proc_icesat_data import proc_data
from make_synth_data import make_data
import os 
if os.path.isfile('../data_'+lake_name+'/h_obs.npy')==False: 
    if lake_name != 'synth' and lake_name !='nonlinear':
        proc_data(lake_name,paths)
    elif lake_name == 'synth':  
        make_data()
    elif lake_name == 'nonlinear':
        print('run fenics code to produce nonlinear synthetic data')     

Next we will run the inversion.

In [ ]:
# example script that runs the inversion
from inversion import invert
import numpy as np
from print_params import print_params

Print the auxiliar parameters (ice thickness, basal drag, viscosity, velocities):

In [ ]:
# print the auxiliary model parameters
print_params()

Invert the data:

In [ ]:
from params import data_dir,Nt

# load elevation data (h_obs)
h_obs = np.load(data_dir+'/h_obs.npy')

# set regularization parameter
eps = 1e0

# set reference time to define elevation anomaly
t_ref = 0 # probably should set t_ref = 1.25 yr for SLM

In [ ]:
# solve for the basal vertical velocity inversion
w_map,h_fwd,mis = invert(h_obs,eps=eps,t_ref=t_ref)

Plot the results:

(the plotting script can be modified in plot_results.py)

In [ ]:
from plot_results import plot
# set the contour plot limits for the elevation and inversion with h_lim and w_lim, resp.
h_lim = 5
w_lim = 5
# timesteps = range(Nt)   # set timesteps to list with length >1 to make a movie
timesteps = [69]        # set timesteps to list with length == 1 to print single snapshot
plot(t_ref,timesteps=timesteps,h_lim=h_lim,w_lim=w_lim)   